## GoDateset

In [1]:
# 环境配置
%cd /playground/sgd_deep_learning/sgd_rl/go
import sys
sys.path.append('./python')

/playground/sgd_deep_learning/sgd_rl


In [2]:
from torch.utils.data import DataLoader
from dlgo.data import GoDataset

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
preprocess_dir = 'data/oneplane/'
num_workers = 0

train_dataset = GoDataset(preprocess_dir, datatype='train')
test_dataset = GoDataset(preprocess_dir, datatype='test')

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=num_workers)

    
# 不shuffle, 就没有很多问题，每次只需要缓存几个文件
# 但是这样就会导致棋局后面的棋面的bias，学习率下降。 所以用 优化器adagrad adadelta
# 可以对比测试一下，shuffle和不shuffle的差异。添加缓存设计后是什么情况。
# reshuffle the data at every epoch to reduce model overfitting

In [4]:
x, y = train_dataset[0]
print(type(x), x.shape, x.dtype)
print(type(y), y.shape, y, y.dtype)

print("数据集样本数", len(train_dataset), len(test_dataset))

<class 'torch.Tensor'> torch.Size([1, 19, 19]) torch.bfloat16
<class 'torch.Tensor'> torch.Size([]) tensor(68) torch.int64
数据集样本数 977920 190464


In [15]:
for x,y in train_dataloader:
    print(x.shape, y.shape)
    break

torch.Size([64, 1, 19, 19]) torch.Size([64])


### 测试dataloader吞吐量

In [5]:
import time

def test_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()  # 记录函数开始时间
        result = func(*args, **kwargs)  # 调用原始函数
        end_time = time.time()  # 记录函数结束时间
        execution_time = end_time - start_time  # 计算函数执行时间
        print(f"{func.__name__} 执行时间: {execution_time:.6f} 秒")
        return result
    return wrapper

@test_time
def load_data_loop(num=10000):
    count = 0
    
    for x, y in train_dataloader:
        if count > num:
            break
        else:
            count += x.shape[0]

### 结果展示
---
    load_data_loop(1e2)
        load_data_loop 执行时间: 1.552093 秒
        每一次都重新打开一遍文件，且只读取一个文件，这个加载效率太低下
    
    增加缓存，减少文件读取次数
        加载1W条数据不到  0.47s << time(gpu内模型计算1W条数据)
        这个dataload 多次执行，还能继续优化加载速度，speedup 0.94s -> 0.47s

In [14]:
load_data_loop(1e4)

load_data_loop 执行时间: 0.473474 秒
